# LightGBMでのタイタニック

## 前処理
## カテゴリ変数の変換


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ls ../input/titanic

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
sample_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
train.head()

In [ ]:
print(train.shape, test.shape, sample_submission.shape)
sample_submission

In [ ]:
# SexとEmbarkedのOne-Hotエンコーディング
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])

# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

# trainの表示
display(train.head())

## 5.2 ホールドアウト法での学習・推論

In [ ]:
X_train = train.drop(['Survived'], axis=1)  # X_trainはtrainのSurvived列以外
y_train = train['Survived']  # Y_trainはtrainのSurvived列

<img src="https://cdn.discordapp.com/attachments/507208726864855060/622684935702183948/5.png" width="600">

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_trainとY_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

# lab.Datasetでtrainとvalidを作っておく
lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(valid_x, valid_y)

# パラメータを定義
lgbm_params = {'objective': 'binary'}

# lgb.trainで学習
evals_result = {}
gbm = lgb.train(params = lgbm_params,
                train_set = lgb_train,
                valid_sets= [lgb_train, lgb_eval],
                early_stopping_rounds=20,
                evals_result=evals_result,
                verbose_eval=10);

In [ ]:
# valid_xについて推論
oof = (gbm.predict(valid_x) > 0.5).astype(int)
print('score', round(accuracy_score(valid_y, oof)*100,2)) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(evals_result['training']['binary_logloss'], label='train_loss')
plt.plot(evals_result['valid_1']['binary_logloss'], label='valid_loss')
plt.legend()

In [ ]:
test_pred = (gbm.predict(test) > 0.5).astype(int)
sample_submission['Survived'] = test_pred
sample_submission.to_csv('train_test_split.csv', index=False) 

## 5.3 3分割交差検証での学習

In [ ]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=0)

# スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):    
    print(f'fold{fold_ + 1} start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]

    # lab.Datasetを使って、trainとvalidを作っておく
    lgb_train= lgb.Dataset(train_x, train_y)
    lgb_valid = lgb.Dataset(valid_x, valid_y)

    # パラメータを定義
    lgbm_params = {'objective': 'binary'}

    # lgb.trainで学習
    gbm = lgb.train(params=lgbm_params,
                    train_set=lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    early_stopping_rounds=20,
                    verbose_eval=-1
                    )
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', round(np.mean(score_list), 2))

In [ ]:
# テストデータの予測を格納する、418行3列のnumpy行列を作成
test_pred = np.zeros((len(test), 3))

for fold_, gbm in enumerate(models):
    test_pred[:, fold_] = gbm.predict(test) # testを予測

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sample_submission['Survived'] = pred
sample_submission.to_csv('3-fold_cross-validation.csv',index=False)